In [102]:
import numpy as np 
import pandas as pd

Có nhiều điều mình không thể hiểu hết về cách đạo hàm của Gradient Descent. <br>
Có những bài đạo hàm phía trái, hoặc đạo hàm phía phải cho kết quả. <br>
Hoặc là sử dụng Numerical Descent lại cho ra cực trị tối ưu - mặc dù không sử dụng momentum...(Mình có code bên dưới) <br>
Bàn luận toán với mình về Gradient Descent <br>

There's many things i can't understand all about how Gradient Descent work. <br>
Some gradient by left, some gradient by right, or using Numerical Descent WITHOUT momentum give us optimal value... (I had code below) <br>
You can help me to improve math in Gradient Descent!

## GRADIENT DESCENT

In [103]:
X = np.random.rand(1000, 1) # random range(0, 1) with shape (1000, 1)
y = X ** 2 + 10 * np.sin(X) + .2 * np.random.randn(1000, 1)

# f(x) = x * w^T (x.shape = (N, feature), w^T.shape = (feature, 1))

GRADIENT

In [104]:
def cost(w: np.ndarray) -> np.ndarray:
    """
    Hàm mục tiêu để tìm minimum point
    """
    return w ** 2 + 10 * np.sin(w)

def grad(w: np.ndarray) -> np.ndarray:
    """
    Đạo hàm bên phải hàm cost 
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_right = w.copy()
            w_right[i, j] += epsilon
            w_grad[i, j] = (cost(w_right) - cost(w)) / epsilon
    return w_grad

def grad_left(w: np.ndarray) -> np.ndarray:
    """
    Đạo hàm bên trái hàm cost 
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_left = w.copy()
            w_left[i, j] -= epsilon
            w_grad[i, j] = (cost(w) - cost(w_left)) / epsilon
    return w_grad

def numerical_grad(w: np.ndarray) -> np.ndarray:
    """
    w(right) - w(left) / 2 * epsilon
    Đạo hàm này là đạo hàm từng phần - đạo hàm 2 phía
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_right = w.copy()
            w_left = w.copy()
            w_right[i, j] += epsilon
            w_left[i, j] -= epsilon
            w_grad[i, j] = (cost(w_right) - cost(w_left)) / (2 * epsilon)
    return w_grad

def check_converged(theta, gradFunc, epsilon=1e-3):
    """
    Kiểm tra sự hội tụ của đạo hàm tại điểm theta
    """
    return np.linalg.norm(gradFunc(theta)) / len(theta) < epsilon

BASIC CONCEPT

In [105]:
def GD(w_init, gradFunc, eta):
    """
        w_init: điểm w khởi tạo, w.init là mảng 2 chiều
        grad: hàm đạo hàm của hàm mục tiêu
        eta: learning rate
    """
    w = [w_init]
    for ep in range(100):
        w_next = w[-1] - eta * gradFunc(w[-1])
        if check_converged(w_next, gradFunc):
            break
        w.append(w_next)
    
    return w, ep

WITH MOMENTUM

$$
v_t = \gamma v_{t-1} + \eta \bigtriangledown_\theta J (\theta)
$$
$$
\theta = \theta - v_t
$$

In [106]:
def GD_momentum(theta_init, gradFunc, eta, gamma=0.9):
    """
    theta_init: like w_init
    eta: learning rate
    gamma: hệ số - thường là 0.9 nhân với v trước
    v_t = gamma * v_t-1 + eta * grad(theta)
    """
    epochs = 5000
    thetas = [theta_init]
    v = np.zeros_like(theta_init)
    for ep in range(epochs):
        v_next = gamma * v + eta * gradFunc(thetas[-1])
        theta_next = thetas[-1] - v_next
        if check_converged(theta_next, gradFunc):
            break
        v = v_next
        thetas.append(theta_next)
        
    return thetas, ep


implement

In [107]:
theta_init = np.array([[5]]) # w phải có kích thước là (feature, 1)
thetas, ep = GD(theta_init, grad, .1)
print('GD - gradient right: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD(theta_init, grad_left, .1)
print('GD - gradient left: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD(theta_init, numerical_grad, .1)
print('GD - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_momentum(theta_init, numerical_grad, .1)
print('GD with momentum - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))

GD - gradient right: w =  [[3.83628779]] 
6 iterations
GD - gradient left: w =  [[-1.30585389]] 
34 iterations
GD - numerical gradient: w =  [[-1.30635179]] 
31 iterations
GD with momentum - numerical gradient: w =  [[-1.30431627]] 
243 iterations


NESTEROV ACCELERATED GRADIENT (NAG)

$$
v_t = \gamma v_{t-1} + \eta \nabla_\theta J (\theta - \gamma v_{t-1})
$$

$$
\theta = \theta - v_t
$$

In [112]:
def GD_nag(theta_init, gradFunc, eta, gamma=0.9):
    epochs = 100
    thetas = [theta_init]
    v = np.zeros_like(theta_init)
    for ep in range(epochs):
        v_next = gamma * v + eta * gradFunc(thetas[-1] - gamma * v)
        theta = thetas[-1] - v_next
        if check_converged(thetas[-1], gradFunc, 1e-3):
            break
        thetas.append(theta)
        v = v_next
    return thetas, ep

In [113]:
theta_init = np.array([[5]]) # w phải có kích thước là (feature, 1)
thetas, ep = GD_nag(theta_init, grad, .1)
print('GD - gradient right: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_nag(theta_init, grad_left, .1)
print('GD - gradient left: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_nag(theta_init, numerical_grad, .1)
print('GD - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))

GD - gradient right: w =  [[5]] 
1 iterations
GD - gradient left: w =  [[-1.30599638]] 
21 iterations
GD - numerical gradient: w =  [[-1.30649585]] 
21 iterations
